##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Apache ORC 리더

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/io/tutorials/orc"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

## 개요

Apache ORC는 인기 있는 열 기반 스토리지 형식입니다. tensorflow-io 패키지는 [Apache ORC](https://orc.apache.org/) 파일을 읽는 기본 구현을 제공합니다.

## 설치

필수 패키지를 설치하고 런타임을 다시 시작하십시오.


In [2]:
!pip install tensorflow-io

In [3]:
import tensorflow as tf
import tensorflow_io as tfio

### ORC로 샘플 데이터세트 파일 다운로드

여기에서 사용할 데이터세트는 UCI의 [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris)입니다. 이 데이터 세트는 각각 50개 인스턴스의 3개 클래스가 포함되며, 각 클래스는 아이리스 식물의 종류를 의미합니다. 여기에는 (1) 꽃받침 길이, (2) 꽃받침 너비, (3) 꽃잎 길이, (4) 꽃잎 너비의 4가지 속성이 있으며 마지막 열은 클래스 레이블을 포함합니다.

In [4]:
!curl -OL https://github.com/tensorflow/io/raw/master/tests/test_orc/iris.orc
!ls -l iris.orc

## 파일에서 데이터세트 생성

In [35]:
dataset = tfio.IODataset.from_orc("iris.orc", capacity=15).batch(1)

데이터세트를 검사합니다.

In [42]:
for item in dataset.take(1):
    print(item)


아이리스 데이터세트를 기반으로 한 ORC 데이터세트로 훈련한 tf.keras 모델의 엔드 투 엔드 예시를 살펴봅니다.

### 데이터 전처리

어떤 열이 기능이고 어떤 열이 레이블인지 다음과 같이 구성합니다.

In [47]:
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
label_cols = ["species"]

# select feature columns
feature_dataset = tfio.IODataset.from_orc("iris.orc", columns=feature_cols)
# select label columns
label_dataset = tfio.IODataset.from_orc("iris.orc", columns=label_cols)

모델 훈련을 위해 종을 실수로 매핑하는 유틸리티 함수는 다음과 같습니다.

In [48]:
vocab_init = tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant(["virginica", "versicolor", "setosa"]),
    values=tf.constant([0, 1, 2], dtype=tf.int64))
vocab_table = tf.lookup.StaticVocabularyTable(
    vocab_init,
    num_oov_buckets=4)

In [49]:
label_dataset = label_dataset.map(vocab_table.lookup)
dataset = tf.data.Dataset.zip((feature_dataset, label_dataset))
dataset = dataset.batch(1)

def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features), axis=1)
    return features, labels

dataset = dataset.map(pack_features_vector)

## 모델 구축, 컴파일 및 훈련하기

마침내 모델을 구축하고 훈련할 준비가 되셨습니다! 방금 처리한 데이터세트에서 아이리스 식물의 클래스를 예측하는 3개의 레이어 keras 모델을 구축합니다.

In [50]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            10, activation=tf.nn.relu, input_shape=(4,)
        ),
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(3),
    ]
)

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.fit(dataset, epochs=5)